## 설치

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 27.8 MB/s 
     |████████████████████████████████| 7.6 MB 72.5 MB/s 
     |████████████████████████████████| 163 kB 91.1 MB/s 


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 27.7 MB/s 
     |████████████████████████████████| 212 kB 85.5 MB/s 
     |████████████████████████████████| 115 kB 84.2 MB/s 
     |████████████████████████████████| 127 kB 86.7 MB/s 
     |████████████████████████████████| 115 kB 74.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 26.8 MB/s 
     |████████████████████████████████| 166 kB 78.0 MB/s 
     |████████████████████████████████| 182 kB 65.3 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 166 kB 69.7 MB/s 
     |████████████████████████████████| 162 kB 71.8 MB/s 
     |████████████████████████████████| 162 kB 69.8 MB/s 
     |████████████████████████████████| 158 kB 60.9 MB/s 
     |████████████████████████████████| 157 kB 70.8 MB/s 
     |████████████████████████████████| 157 kB 72.1 MB/s 
     |████████████████████████████████| 157 kB 72.4 MB/s 
     |████████████████████████████████| 157 kB 89.0 MB/s 
     |████████████████████████████████| 157 kB 71.0 MB/s 
     |████████████████████████████████| 157 kB 71.3 MB/s 
     |████████████████████████████████| 157 kB 72.0 MB/s 
     |████████████████████████████████| 156 kB 88.4 MB/s 


In [4]:
!kaggle datasets download -d minjaechoi99/aihub-groom

 87% 33.0M/38.1M [00:02<00:00, 12.6MB/s]
100% 38.1M/38.1M [00:02<00:00, 13.7MB/s]


In [5]:
!unzip /content/aihub-groom.zip

Archive:  /content/aihub-groom.zip
  inflating: data/Training/talksets-train-1.json  
  inflating: data/Training/talksets-train-2.json  
  inflating: data/Training/talksets-train-3.json  
  inflating: data/Training/talksets-train-4.json  
  inflating: data/Training/talksets-train-5.json  
  inflating: data/Validation/talksets-train-6.json  


## readjson

In [1]:
import torch
import torch.optim as optim

import numpy as np
from tqdm import tqdm as tqdm

from transformers import(
    AutoModel,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    default_data_collator,
    DefaultDataCollator,
    EarlyStoppingCallback,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    get_constant_schedule,
    AdamW
)

In [2]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mjchoi. Use `wandb login --relogin` to force relogin


True

In [3]:
from pathlib import Path
import json, re
def read_aihub(path):
    path = Path(path)
    with open(path, 'rb') as f:
        print(f)
        squad_dict = json.load(f)
    texts = []
    ids = []
    for group in squad_dict:
        for passage in group['sentences']:
            id = passage['id']
            text = passage['origin_text']
            text = re.sub('#.*?#', '[UNK]', text)
            intensity = passage['intensity']
            if intensity > 1:
                texts.append(text)
                ids.append(id)

    return texts, ids

In [4]:
import pandas as pd

texts = []
ids = []
for i in range(1,2):
    file_path =  f'/content/data/Training/talksets-train-{i}.json'
    ttexts, iids = read_aihub(file_path)
    texts += ttexts
    ids += iids
immoral_df = pd.DataFrame(list(texts),
                        columns =['texts'])

<_io.BufferedReader name='/content/data/Training/talksets-train-1.json'>


In [ ]:
immoral_df

In [6]:
from datasets import Dataset

immoral_data = Dataset.from_pandas(immoral_df)

In [7]:
immoral_data

Dataset({
    features: ['texts'],
    num_rows: 23628
})

## tokenize

In [8]:
model_name = 'beomi/KcELECTRA-base-v2022'
model_version = 'groom2team/pj3_class_kcelectra/pytorch_finetuned:v8'
max_length = 128

In [9]:
import wandb
run = wandb.init()
artifact1 = run.use_artifact(model_version, type='model')
artifact_dir1 = artifact1.download()

model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.load_state_dict(torch.load(artifact_dir1+'/best_model_at_end/pytorch_model.bin'))

tokenizer = AutoTokenizer.from_pretrained(model_name)

wandb: Downloading large artifact pytorch_finetuned:v8, 489.19MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:0.1
Some weights of the model checkpoint at beomi/KcELECTRA-base-v2022 were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were

In [10]:
def tokenizeWithoutLabel(data):
    tokenized_datas = tokenizer(
        data['texts'],
        max_length=max_length,
        padding="max_length",
        truncation="only_second"
    )
    return tokenized_datas

In [11]:
device = torch.device('cuda')
model = model.to(device)

In [12]:
torch.cuda.empty_cache()

In [13]:
def del_list_index(tokens, indexs):
    out = tokens[:]
    indexs = list(indexs)
    for index in indexs[::-1]:
        if out[index] == '[UNK]':
            continue
        del out[index]
    return out

In [14]:
from datasets.utils import disable_progress_bar
disable_progress_bar()

In [43]:
from itertools import combinations
def delete_style_token(text):
    texts = []
    texts.append({'ids': [0], 'texts' : text})
    tokens = tokenizer.encode(text)
    token_indexs = range(1, len(tokens) - 1)
    for n in range(1,4):
        if len(token_indexs) < n - 1:
            continue
        for indexs in combinations(token_indexs, n):
            texts.append({'ids' : indexs, 'texts' : tokenizer.decode(del_list_index(tokens, indexs)[1:-1])})
    line_data = Dataset.from_list(texts)
    line_tokenized_datasets = line_data.map(tokenizeWithoutLabel, batched=True, remove_columns=line_data.column_names, )
    data_collator = DefaultDataCollator(return_tensors="pt")
    line_loader = torch.utils.data.DataLoader(line_tokenized_datasets, batch_size=32,
                                            shuffle=False, collate_fn=data_collator,
                                            num_workers=0)
    max_diff = 0
    max_info = {}
    vanil_prob = 0
    for up_i, id_inputs in enumerate(line_loader):
        inputs = {}
        inputs['input_ids'] = id_inputs['input_ids'].to(device)
        inputs['token_type_ids'] = id_inputs['token_type_ids'].to(device)
        inputs['attention_mask'] = id_inputs['attention_mask'].to(device)
        outputs = model(**inputs)
        logits = outputs.logits
        softmax = logits.softmax(dim=-1)
        for i, prob in enumerate(softmax):
            if up_i == 0 and i == 0:
                if prob[1] < 0.7:
                    return 
                else:
                    vanil_prob = prob
            else:
                if vanil_prob[1] - prob[1] > max_diff:
                    max_info = (texts[i], prob)
                    max_diff = vanil_prob[1] - prob[1]
    print(max_info)
    if max_diff >= 0.2:
        return {'masked':max_info[0]['texts'], 'original':text}
            

## eval

In [16]:
eval_model_name = 'skt/kogpt2-base-v2'
eval_model_version = 'groom2team/pj3_gen_gpt2/pytorch_finetuned:v0'

In [17]:
run = wandb.init()
artifact2 = run.use_artifact(eval_model_version, type='model')
artifact_dir2 = artifact2.download()

eval_model = AutoModelForCausalLM.from_pretrained(eval_model_name)
eval_model.load_state_dict(torch.load(artifact_dir2+'/best_model_at_end/pytorch_model.bin'))

wandb: Downloading large artifact pytorch_finetuned:v0, 492.98MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:0.1


<All keys matched successfully>

In [18]:
gpttokenizer = AutoTokenizer.from_pretrained(eval_model_name,
                                          bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                          pad_token='<pad>', mask_token='<mask>')

In [37]:
def tokenizeMasked(data):
    text = delete_style_token(data)
    print(text)
    tokenized_datas = gpttokenizer(
        f"<unused0> <unused1> {text['masked']} <unused2>",
        return_tensors="pt"
    )
    return tokenized_datas

In [ ]:
input = tokenizeMasked('같은여자로 존나 창피하다먹고 살려고 하는건가')
gen_ids = eval_model.generate(**input,
                         max_length=256,
                         pad_token_id=gpttokenizer.pad_token_id,
                         eos_token_id=gpttokenizer.eos_token_id,
                         bos_token_id=gpttokenizer.bos_token_id)
gpttokenizer.decode(gen_ids[0])